In [7]:
import torch
torch.cuda.is_available()

True

In [8]:
import open_clip

clip_model_pretrain, _, _ = open_clip.create_model_and_transforms('MobileCLIP-S2', pretrained='datacompdr')
clip_model_pretrain.cuda()

CustomTextCLIP(
  (visual): TimmModel(
    (trunk): FastVit(
      (stem): Sequential(
        (0): MobileOneBlock(
          (se): Identity()
          (conv_kxk): ModuleList(
            (0): ConvNormAct(
              (conv): Conv2d(3, 80, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (bn): BatchNormAct2d(
                80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
                (drop): Identity()
                (act): Identity()
              )
            )
          )
          (conv_scale): ConvNormAct(
            (conv): Conv2d(3, 80, kernel_size=(1, 1), stride=(2, 2), bias=False)
            (bn): BatchNormAct2d(
              80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): Identity()
            )
          )
          (act): GELU(approximate='none')
        )
        (1): MobileOneBlock(
          (se): Identity()
          (conv_kxk): ModuleList

In [9]:
for _, p in enumerate(clip_model_pretrain.text.parameters()):
    p.requires_grad_(False)

In [11]:
from torch import nn

visual = nn.Sequential(
    nn.Linear(40, 512, bias=True),
    nn.LayerNorm(512),
    nn.Mish(inplace=True),
    nn.TransformerEncoder(
        nn.TransformerEncoderLayer(d_model=512, nhead=8),
        num_layers=4,
        norm=nn.LayerNorm(512),
    )
).cuda()

d:\Repos\clip-bold\.venv\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [12]:
clip_model_pretrain.visual = visual

In [13]:
import mobileclip

tokenizer = mobileclip.get_tokenizer('mobileclip_s2')
tokenizer.cuda()

d:\Repos\clip-bold\.venv\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


ClipTokenizer()

In [14]:
text = tokenizer(["a brain", "a dog", "a cat"])
text

tensor([[49406,   320,  4812, 49407,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [49406,   320,  1929, 49407,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0, 

In [15]:
clip_model_pretrain.encode_text(text.cuda())

tensor([[ 0.1200, -0.4782,  0.0519,  ...,  0.3060,  0.3293, -0.3102],
        [-0.0275, -0.1664,  0.1085,  ...,  0.2483, -0.0473, -0.6230],
        [ 0.0829, -0.3982,  0.0995,  ...,  0.1728,  0.1667, -0.2993]],
       device='cuda:0')

In [16]:
img = torch.rand((3, 40)).cuda()
clip_model_pretrain.encode_image(img)

tensor([[-0.5539, -2.0685, -0.6479,  ...,  0.0858,  1.7285,  0.9136],
        [-0.5212, -1.7529, -1.7420,  ..., -0.2306,  1.4483,  1.3964],
        [-0.3348, -2.0048, -1.7319,  ...,  0.5718,  1.3747,  1.2085]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)

In [17]:
image_features, text_features, logit_scale = clip_model_pretrain.forward(image=img, text=text.cuda())

In [18]:
clip_model_pretrain.logit_scale

Parameter containing:
tensor(4.1374, device='cuda:0', requires_grad=True)

In [19]:
logits_per_image = logit_scale * image_features @ text_features.T
logits_per_text = logits_per_image.T

logits_per_image.shape

torch.Size([3, 3])

In [20]:
logits_per_image

tensor([[-0.5650, -1.6828, -0.4546],
        [ 0.3379, -1.4314, -0.4752],
        [-0.2605, -1.6835,  0.0026]], device='cuda:0', grad_fn=<MmBackward0>)

In [21]:
labels = torch.arange(logits_per_image.shape[0], device='cuda', dtype=torch.long)
labels

tensor([0, 1, 2], device='cuda:0')

In [23]:
import torch.nn.functional as F

F.cross_entropy(logits_per_image, labels)

tensor(1.2704, device='cuda:0', grad_fn=<NllLossBackward0>)

In [24]:
F.cross_entropy(logits_per_text, labels)

tensor(1.1079, device='cuda:0', grad_fn=<NllLossBackward0>)

In [25]:
import torch.nn.functional as F

total_loss = (
    F.cross_entropy(logits_per_image, labels) +
    F.cross_entropy(logits_per_text, labels)
) / 2
total_loss

tensor(1.1892, device='cuda:0', grad_fn=<DivBackward0>)

: 

In [12]:
open_clip.model.CustomTextCLIP

open_clip.model.CustomTextCLIP